In [1]:
import pandas as pd

In [2]:
inputs = pd.read_csv('./data/inputs.csv', header=None, names=['id','tx_id','sig_id','output_id'])
transactions = pd.read_csv('./data/transactions.csv', header=None, names=['tx_id','block_id'])
outputs = pd.read_csv('./data/outputs.csv', header=None, names=['id','tx_id','pk_id','value'])

In [3]:
invalid_transactions = set()

Look for the double spending transactions:

In [ ]:
double_spending_transactions = inputs[(inputs['output_id'].duplicated(keep='first')) & (inputs['output_id']!=-1)][['tx_id','output_id']]
invalid_transactions.update(double_spending_transactions['tx_id'])
double_spending_transactions

The double spending transactions are added to the invalid transactions list

In [5]:
for _,transaction in double_spending_transactions.iterrows():
    invalid_transactions.add(transaction['tx_id'])

Now we have to check that the sum(inputs) >= sum(outputs)
First we get a view in which we have each transaction associated with its input amount

In [6]:
inputs_with_value = pd.merge(inputs, outputs, left_on='output_id', right_on='id')
inputs_with_value.drop(columns=['output_id','pk_id','sig_id','id_x','id_y','tx_id_y'], inplace=True)
inputs_with_value.rename(columns={'tx_id_x':'transaction_A'}, inplace=True)

This dataframes show the input amount of each transaction

In [7]:
inputs_with_value = inputs_with_value.groupby('transaction_A',as_index=False).sum()
inputs_with_value = inputs_with_value.rename(columns={'value':'input_value'})
inputs_with_value

,transaction_A,input_value
0,172,5000000000
1,184,4000000000
2,186,3000000000
3,188,2900000000
4,193,100000000
...,...,...
116602,216618,13836000000
116603,216622,5000000000
116604,216623,300000000
116605,216624,1000000


let's do the same for the outputs:

In [8]:
transactions_balance = pd.merge(inputs_with_value, outputs, left_on='transaction_A', right_on='tx_id')
transactions_balance = transactions_balance.drop(columns=['pk_id','transaction_A','id',])
transactions_balance = transactions_balance.rename(columns={'value':'output_value'})
transactions_balance = transactions_balance.groupby(['tx_id','input_value'], as_index=False)['output_value'].sum()

In [9]:
pd.merge(inputs_with_value, transactions, left_on='transaction_A', right_on='tx_id')

,transaction_A,input_value,tx_id,block_id
0,172,5000000000,172,170
1,184,4000000000,184,181
2,186,3000000000,186,182
3,188,2900000000,188,183
4,193,100000000,193,187
...,...,...,...,...
116602,216618,13836000000,216618,100013
116603,216622,5000000000,216622,100016
116604,216623,300000000,216623,100016
116605,216624,1000000,216624,100016


The last thing to do is to check if the output is bigger than the input

In [1]:
transactions_balance = transactions_balance[transactions_balance['input_value'] < transactions_balance['output_value']]
transactions_balance

NameError: name 'transactions_balance' is not defined

In [11]:
for _,transaction in transactions_balance.iterrows():
    invalid_transactions.add(transaction['tx_id'])

Now we are going to check if there's some transaction where the output value is not >= 0:

In [12]:
negative_output = outputs[outputs['value'] < 0]
print(negative_output)
for _,transaction in negative_output.iterrows():
    invalid_transactions.add(transaction['tx_id'])

            id   tx_id   pk_id       value
123671  123672  105281  155098 -5000000000


Now we are going to see if for every input transaction there exist at least one output transaction

In [13]:
inputs_tx_id  = inputs.groupby('tx_id', as_index=False).sum()['tx_id']
outputs_tx_id = outputs.groupby('tx_id', as_index=False).sum()['tx_id']
pd.concat([inputs_tx_id,outputs_tx_id]).drop_duplicates(keep=False)

Series([], Name: tx_id, dtype: int64)

This result shows that each input transaction has at least one output transaction.

Removing Invalid value for the outputs

Checking if the sig_id of the inputs transaction correspond to the pk_id of the transaction where the output belongs

In [14]:
signature_check =  pd.merge(inputs, outputs, left_on='output_id', right_on='id')
signature_check = signature_check.drop(columns=['id_x','tx_id_y','output_id'])
print(signature_check[signature_check['sig_id'] != signature_check['pk_id']])

invalid_transactions.add(138278)
invalid_transactions
#Transaction where the output_id referenced doesn't exixst:
invalid_transactions.add(265834)

        tx_id_x  sig_id    id_y   pk_id       value
77265    116411      -1  137338  113300  5000000000
104729   138278  139250   16121   16020  5000000000


There're 2 nonvalid signatures. Since sig_id=-1 means that a non-standard script has been used, the only one to be removed is the tx_id=138278

Now we are going to remove from the dataframe "transactions" all the transactions which don't have a counterpart in inputs/ouputs.
Since we've showed that for every input there's a corresponding output we are going to perform the check only on one dataframe.

In [15]:
inputs_tx_id  = inputs.groupby('tx_id', as_index=False).sum()['tx_id']
transactions_tx_id = transactions.groupby('tx_id', as_index=False).sum()['tx_id']
inputs_tx_id.shape[0]
invalid_transactions
invalid_transactions.remove(207365)

{12152, 30446, 61845, 100929, 105281, 138278, 207365, 265834}

Those two cells find the chain of transactions linked to the invalid ones

In [16]:
cb_transactions = inputs[inputs['output_id']==-1]
cb_transaction_output = pd.merge(cb_transactions,outputs, left_on='tx_id', right_on='tx_id').drop(columns=['id_x','sig_id','output_id','id_y','pk_id'])
cb_transaction_output = cb_transaction_output.groupby('tx_id', as_index=False).sum()
cb_transaction_output[cb_transaction_output['value'] < 5000000000]
invalid_transactions.update(cb_transaction_output[cb_transaction_output['value'] < 5000000000]['tx_id'])

In [17]:
flag = False
transactions_to_check = pd.Series(list(invalid_transactions)).rename('tx_id')
while not flag:
    #Take the ID of the invalid transactions:
    previous_len = len(invalid_transactions)
    invalid_transactions_id = (pd.merge(transactions_to_check, outputs, left_on='tx_id', right_on='tx_id')['id']).rename('id')
    #Check if the invalid transactions ID has been used in other transactions, and take those transactions tx_id
    transactions_to_check = pd.merge(invalid_transactions_id, inputs, left_on='id', right_on='output_id').drop_duplicates()['tx_id']
    print("Invalid_tx_id: {} \n Transactions_to_check: {}".format(invalid_transactions_id.shape,transactions_to_check.shape))
    invalid_transactions.update(transactions_to_check)
    flag = previous_len == len(invalid_transactions)
    print("Actual invalid transactions: {}".format(len(invalid_transactions)))

Invalid_tx_id: (14,) 
 Transactions_to_check: (2,)
Actual invalid transactions: 12
Invalid_tx_id: (3,) 
 Transactions_to_check: (1,)
Actual invalid transactions: 13
Invalid_tx_id: (2,) 
 Transactions_to_check: (2,)
Actual invalid transactions: 15
Invalid_tx_id: (3,) 
 Transactions_to_check: (2,)
Actual invalid transactions: 17
Invalid_tx_id: (3,) 
 Transactions_to_check: (1,)
Actual invalid transactions: 18
Invalid_tx_id: (2,) 
 Transactions_to_check: (1,)
Actual invalid transactions: 19
Invalid_tx_id: (2,) 
 Transactions_to_check: (1,)
Actual invalid transactions: 20
Invalid_tx_id: (2,) 
 Transactions_to_check: (2,)
Actual invalid transactions: 22
Invalid_tx_id: (3,) 
 Transactions_to_check: (2,)
Actual invalid transactions: 24
Invalid_tx_id: (3,) 
 Transactions_to_check: (1,)
Actual invalid transactions: 25
Invalid_tx_id: (2,) 
 Transactions_to_check: (2,)
Actual invalid transactions: 27
Invalid_tx_id: (3,) 
 Transactions_to_check: (1,)
Actual invalid transactions: 28
Invalid_tx_id: 

KeyboardInterrupt: 

In [ ]:
#tmp = pd.DataFrame(list(invalid_transactions), columns=['tx_id'])
#tmp.to_csv("./data/invalid_transactions.csv")
#tmp

In [ ]:
invalid_transactions = pd.read_csv("data/invalid_transactions.csv", index_col=0).squeeze()
invalid_transactions

Delete all of the invalid transactions

In [ ]:
print("Before: a= {}, b= {}, c={}".format(inputs.shape[0],transactions.shape[0],outputs.shape[0]))
print("Number of invalid transaction : {}".format(len(invalid_transactions)))
for value in invalid_transactions:
    inputs.drop(inputs[inputs['tx_id'] == value].index, inplace=True)
    transactions.drop(transactions[transactions['tx_id'] == value].index, inplace=True)
    outputs.drop(outputs[outputs['tx_id'] == value].index, inplace=True)
print("After: a= {}, b= {}, c={}".format(inputs.shape[0],transactions.shape[0],outputs.shape[0]))

In [ ]:
cond1 = inputs['tx_id'].isin(invalid_transactions)
cond2 = transactions['tx_id'].isin(invalid_transactions)
cond3 = outputs['tx_id'].isin(invalid_transactions)
df1 = inputs.drop(inputs[cond1].index)
df2 = transactions.drop(transactions[cond2].index)
df3 = outputs.drop(outputs[cond3].index)

In [ ]:
df1.to_csv("./data/updated_inputs.csv")
df2.to_csv("./data/updated_transactions.csv")
df3.to_csv("././data/updated_outputs.csv")